In [125]:
import os

import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 300)

In [280]:
DATA_DIRECTORY = '../data'
year = 2020
PBP_PATH = os.path.join(DATA_DIRECTORY, "play_by_play", f'{year}.csv')

In [282]:
df = pd.read_csv(PBP_PATH, low_memory=False)
df['year'] = year
df.shape

(2900, 341)

In [283]:
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,...,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,fumbled_1_team,fumbled_1_player_id,fumbled_1_player_name,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,pla

In [131]:
# check with id column to join on
#  'passer_id','rusher_id','receiver_id'
id_cols = [col for col in df.columns if "_id" in col]
id_cols

['play_id',
 'game_id',
 'old_game_id',
 'passer_player_id',
 'receiver_player_id',
 'rusher_player_id',
 'lateral_receiver_player_id',
 'lateral_rusher_player_id',
 'lateral_sack_player_id',
 'interception_player_id',
 'lateral_interception_player_id',
 'punt_returner_player_id',
 'lateral_punt_returner_player_id',
 'kickoff_returner_player_id',
 'lateral_kickoff_returner_player_id',
 'punter_player_id',
 'kicker_player_id',
 'own_kickoff_recovery_player_id',
 'blocked_player_id',
 'tackle_for_loss_1_player_id',
 'tackle_for_loss_2_player_id',
 'qb_hit_1_player_id',
 'qb_hit_2_player_id',
 'forced_fumble_player_1_player_id',
 'forced_fumble_player_2_player_id',
 'solo_tackle_1_player_id',
 'solo_tackle_2_player_id',
 'assist_tackle_1_player_id',
 'assist_tackle_2_player_id',
 'assist_tackle_3_player_id',
 'assist_tackle_4_player_id',
 'pass_defense_1_player_id',
 'pass_defense_2_player_id',
 'fumbled_1_player_id',
 'fumbled_2_player_id',
 'fumble_recovery_1_player_id',
 'fumble_recove

In [285]:
df_rush_pbp = df.loc[
    df['play_type'].isin(['run', 'qb_kneel'])
    & (df['sack'] == 0)
    & (df["two_point_attempt"] == 0) 
]

## Designed Run Yards (includes some QBs)
- This does not include scrambles
- We get rid of qb kneeels

In [287]:
cols = ["year", "week",
        "rusher_id", "rusher", 
        "rush", "yards_gained", "rush_touchdown", "fumble", "fumble_lost", "fumble_out_of_bounds"]

mask = df_rush_pbp['play_type'] != 'qb_kneel'
df_rush_stats_designed = df_rush_pbp[mask][cols].groupby(["year", "week", "rusher_id", "rusher"], as_index=False).sum()
df_rush_stats_designed.sort_values("yards_gained", ascending=False)

,year,week,rusher_id,rusher,rush,yards_gained,rush_touchdown,fumble,fumble_lost,fumble_out_of_bounds
122,2020,1,32013030-2d30-3033-3633-3630b28b4868,C.Edwards-Helaire,25,138.0,1.0,0.0,0.0,0.0
40,2020,1,32013030-2d30-3033-3237-3634214e1686,D.Henry,31,116.0,0.0,0.0,0.0,0.0
95,2020,1,32013030-2d30-3033-3532-3137ad078b2e,B.Snell,19,113.0,0.0,1.0,0.0,0.0
44,2020,1,32013030-2d30-3033-3330-343523cdc4a0,E.Elliott,22,96.0,1.0,0.0,0.0,0.0
47,2020,1,32013030-2d30-3033-3332-3830ec2c3be7,C.McCaffrey,23,96.0,2.0,0.0,0.0,0.0
5,2020,1,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14,93.0,0.0,0.0,0.0,0.0
107,2020,1,32013030-2d30-3033-3537-3030adfaf832,J.Jacobs,25,93.0,3.0,0.0,0.0,0.0
61,2020,1,32013030-2d30-3033-3336-39396b8ab7d5,A.Ekeler,19,84.0,0.0,0.0,0.0,0.0
29,2020,1,32013030-2d30-3033-3138-30361fd87e06,M.Brown,18,79.0,2.0,0.0,0.0,0.0
31,2020,1,32013030-2d30-3033-3231-343457f8448b,M.Gordon,15,78.0,1.0,0.0,0.0,0.0


## Scramble Run Yards

In [289]:
cols = ["year", "week",
    "passer_id", "passer", 'pass', "yards_gained", "rush_touchdown", "fumble", "fumble_lost", "fumble_out_of_bounds"]

df_rush_stats_scrambles = df_rush_pbp[cols].groupby(["year", "week","passer_id","passer"], as_index=False).sum()
df_rush_stats_scrambles.sort_values("yards_gained", ascending=False)

,year,week,passer_id,passer,pass,yards_gained,rush_touchdown,fumble,fumble_lost,fumble_out_of_bounds
21,2020,1,32013030-2d30-3033-3532-323871689b29,K.Murray,7,90.0,1.0,0.0,0.0,0.0
9,2020,1,32013030-2d30-3032-3936-303492e9d55e,K.Cousins,4,34.0,0.0,0.0,0.0,0.0
13,2020,1,32013030-2d30-3033-3330-37372d13d088,D.Prescott,3,30.0,0.0,0.0,0.0,0.0
20,2020,1,32013030-2d30-3033-3438-3537f3a13a66,J.Allen,3,28.0,0.0,0.0,0.0,0.0
16,2020,1,32013030-2d30-3033-3335-33372bbab8b8,D.Watson,4,27.0,0.0,0.0,0.0,0.0
11,2020,1,32013030-2d30-3033-3132-3337a759b614,T.Bridgewater,4,26.0,0.0,0.0,0.0,0.0
17,2020,1,32013030-2d30-3033-3338-363967f6a360,M.Trubisky,2,26.0,0.0,0.0,0.0,0.0
25,2020,1,32013030-2d30-3033-3537-31306de02154,D.Jones,3,23.0,0.0,0.0,0.0,0.0
23,2020,1,32013030-2d30-3033-3532-3839c25a936a,G.Minshew II,4,22.0,0.0,0.0,0.0,0.0
5,2020,1,32013030-2d30-3032-3634-39382e08c011,M.Stafford,4,22.0,0.0,0.0,0.0,0.0


## QB Kneel Yards

In [290]:
cols = ["year", "week","rusher_id", "rusher", "qb_kneel", "yards_gained"]

mask = df_rush_pbp['play_type'] == 'qb_kneel'
df_rush_stats_kneels = df_rush_pbp[mask][cols].groupby(["year", "week","rusher_id", "rusher"], as_index=False).sum()
df_rush_stats_kneels.sort_values("yards_gained", ascending=False)

,year,week,rusher_id,rusher,qb_kneel,yards_gained
0,2020,1,32013030-2d30-3032-3035-33318e1e348d,D.Brees,1,-1.0
2,2020,1,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1,-1.0
3,2020,1,32013030-2d30-3032-3739-33399bbb0097,C.Newton,1,-1.0
4,2020,1,32013030-2d30-3032-3831-3138b775ef66,T.Taylor,1,-1.0
5,2020,1,32013030-2d30-3032-3932-363323eefb5c,R.Wilson,1,-1.0
6,2020,1,32013030-2d30-3032-3936-3635a9a33dbc,R.Griffin III,1,-1.0
7,2020,1,32013030-2d30-3033-3331-30361d564889,J.Goff,1,-1.0
8,2020,1,32013030-2d30-3033-3335-33372bbab8b8,D.Watson,1,-1.0
10,2020,1,32013030-2d30-3033-3438-3535737060f8,B.Mayfield,1,-1.0
15,2020,1,32013030-2d30-3033-3537-31306de02154,D.Jones,1,-1.0


## Sack Yards

In [292]:
df_sack_pbp = df.loc[
    (df['play_type'] == 'pass')
    & (df['sack'] == 1)
    & (df["two_point_attempt"] == 0) 
]
df_sack_pbp.shape

(68, 341)

In [293]:
cols = ["year", "week","passer_id", "passer", "sack", "yards_gained"]

df_rush_stats_sacks = df_sack_pbp[cols].groupby(["year", "week","passer_id", "passer"], as_index=False).sum()
df_rush_stats_sacks.sort_values("yards_gained", ascending=False)

,year,week,passer_id,passer,sack,yards_gained
7,2020,1,32013030-2d30-3032-3831-3138b775ef66,T.Taylor,2.0,-1.0
10,2020,1,32013030-2d30-3032-3937-3031bbb58137,R.Tannehill,1.0,-2.0
16,2020,1,32013030-2d30-3033-3331-3139ba7e51e2,J.Brissett,1.0,-6.0
22,2020,1,32013030-2d30-3033-3438-3537f3a13a66,J.Allen,3.0,-6.0
15,2020,1,32013030-2d30-3033-3331-30361d564889,J.Goff,1.0,-6.0
24,2020,1,32013030-2d30-3033-3532-323871689b29,K.Murray,2.0,-6.0
19,2020,1,32013030-2d30-3033-3338-3733fa30c4fa,P.Mahomes,1.0,-8.0
1,2020,1,32013030-2d30-3032-3035-33318e1e348d,D.Brees,1.0,-9.0
3,2020,1,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1.0,-9.0
5,2020,1,32013030-2d30-3032-3634-39382e08c011,M.Stafford,1.0,-9.0


## Putting them all together

In [160]:
df_rush_stats_designed.head()

,rusher_id,rusher,rush,yards_gained,rush_touchdown,fumble,fumble_lost,fumble_out_of_bounds
0,32013030-2d30-3031-3935-39361b587621,T.Brady,2,5.0,1.0,0.0,0.0,0.0
1,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1,0.0,0.0,1.0,0.0,0.0
2,32013030-2d30-3032-3334-35395dc60da5,A.Rodgers,1,2.0,0.0,0.0,0.0,0.0
3,32013030-2d30-3032-3335-3030dd77cad3,F.Gore,6,24.0,0.0,0.0,0.0,0.0
4,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1,2.0,0.0,0.0,0.0,0.0


In [161]:
df_rush_stats_scrambles.head()

,passer_id,passer,pass,yards_gained,rush_touchdown,fumble,fumble_lost,fumble_out_of_bounds
0,32013030-2d30-3031-3935-39361b587621,T.Brady,1,4.0,0.0,0.0,0.0,0.0
1,32013030-2d30-3032-3035-33318e1e348d,D.Brees,1,1.0,0.0,0.0,0.0,0.0
2,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1,11.0,0.0,0.0,0.0,0.0
3,32013030-2d30-3032-3239-34322bc30020,P.Rivers,1,3.0,0.0,0.0,0.0,0.0
4,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,3,17.0,0.0,0.0,0.0,0.0


In [162]:
# df_rush_stats_scrambles.rename(columns={
#     **{col: 'scramble_' + col for col in list(df_rush_stats_scrambles.columns) if col not in ('passer', 'pass')},
#     'passer': 'rusher' 
# }, inplace=True)

# df_rush_stats_scrambles.head()

In [187]:
df_rush_stats_all = df_rush_stats_designed.merge(
    df_rush_stats_scrambles, how='outer', left_on="rusher_id", right_on='passer_id', suffixes=['_designed', '_scrambles']
)

df_rush_stats_all['player'] = np.where(df_rush_stats_all['rusher'].isna(), df_rush_stats_all['passer'], df_rush_stats_all['rusher'])
df_rush_stats_all['player_id'] = np.where(df_rush_stats_all['rusher_id'].isna(), df_rush_stats_all['passer_id'], df_rush_stats_all['rusher_id'])

assert sum(df_rush_stats_all['player'].isna()) == 0
assert sum(df_rush_stats_all['player_id'].isna()) == 0

df_rush_stats_all

,rusher_id,rusher,rush,yards_gained_designed,rush_touchdown_designed,fumble_designed,fumble_lost_designed,fumble_out_of_bounds_designed,passer_id,passer,pass,yards_gained_scrambles,rush_touchdown_scrambles,fumble_scrambles,fumble_lost_scrambles,fumble_out_of_bounds_scrambles,player,player_id
0,32013030-2d30-3031-3935-39361b587621,T.Brady,2.0,5.0,1.0,0.0,0.0,0.0,32013030-2d30-3031-3935-39361b587621,T.Brady,1.0,4.0,0.0,0.0,0.0,0.0,T.Brady,32013030-2d30-3031-3935-39361b587621
1,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1.0,0.0,0.0,1.0,0.0,0.0,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1.0,11.0,0.0,0.0,0.0,0.0,B.Roethlisberger,32013030-2d30-3032-3239-3234f41ff7f7
2,32013030-2d30-3032-3334-35395dc60da5,A.Rodgers,1.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.Rodgers,32013030-2d30-3032-3334-35395dc60da5
3,32013030-2d30-3032-3335-3030dd77cad3,F.Gore,6.0,24.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F.Gore,32013030-2d30-3032-3335-3030dd77cad3
4,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1.0,2.0,0.0,0.0,0.0,0.0,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,3.0,17.0,0.0,0.0,0.0,0.0,R.Fitzpatrick,32013030-2d30-3032-3336-3832b8755c76
5,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14.0,93.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.Peterson,32013030-2d30-3032-3533-3934ceb6c025
6,32013030-2d30-3032-3631-34338634c9b6,M.Ryan,1.0,-1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M.Ryan,32013030-2d30-3032-3631-34338634c9b6
7,32013030-2d30-3032-3634-39382e08c011,M.Stafford,1.0,1.0,0.0,0.0,0.0,0.0,32013030-2d30-3032-3634-39382e08c011,M.Stafford,4.0,22.0,0.0,0.0,0.0,0.0,M.Stafford,32013030-2d30-3032-3634-39382e08c011
8,32013030-2d30-3032-3731-35304d0e9eb8,J.Edelman,1.0,23.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J.Edelman,32013030-2d30-3032-3731-35304d0e9eb8
9,32013030-2d30-3032-3739-33399bbb0097,C.Newton,13.0,64.0,2.0,0.0,0.0,0.0,32013030-2d30-3032-3739-33399bbb0097,C.Newton,1.0,12.0,0.0,0.0,0.0,0.0,C.Newton,32013030-2d30-3032-3739-33399bbb0097


In [188]:
#df_rush_stats_kneels.rename(columns={'yards_gained': 'qb_kneel_yards_gained'}, inplace=True)
df_rush_stats_kneels

,rusher_id,rusher,qb_kneel,yards_gained
0,32013030-2d30-3032-3035-33318e1e348d,D.Brees,1,-1.0
1,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1,-2.0
2,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1,-1.0
3,32013030-2d30-3032-3739-33399bbb0097,C.Newton,1,-1.0
4,32013030-2d30-3032-3831-3138b775ef66,T.Taylor,1,-1.0
5,32013030-2d30-3032-3932-363323eefb5c,R.Wilson,1,-1.0
6,32013030-2d30-3032-3936-3635a9a33dbc,R.Griffin III,1,-1.0
7,32013030-2d30-3033-3331-30361d564889,J.Goff,1,-1.0
8,32013030-2d30-3033-3335-33372bbab8b8,D.Watson,1,-1.0
9,32013030-2d30-3033-3431-37377f312969,T.Boyle,2,-2.0


In [189]:
df_rush_stats_all = df_rush_stats_all.merge(
    df_rush_stats_kneels, how='outer', left_on="player_id", right_on='rusher_id', suffixes=['', '_kneel']
)

df_rush_stats_all['player'] = np.where(df_rush_stats_all['player'].isna(), df_rush_stats_all['rusher_kneel'], df_rush_stats_all['player'])
df_rush_stats_all['player_id'] = np.where(df_rush_stats_all['player_id'].isna(), df_rush_stats_all['rusher_id_kneel'], df_rush_stats_all['player_id'])

assert sum(df_rush_stats_all['player'].isna()) == 0
assert sum(df_rush_stats_all['player_id'].isna()) == 0

df_rush_stats_all

,rusher_id,rusher,rush,yards_gained_designed,rush_touchdown_designed,fumble_designed,fumble_lost_designed,fumble_out_of_bounds_designed,passer_id,passer,pass,yards_gained_scrambles,rush_touchdown_scrambles,fumble_scrambles,fumble_lost_scrambles,fumble_out_of_bounds_scrambles,player,player_id,rusher_id_kneel,rusher_kneel,qb_kneel,yards_gained
0,32013030-2d30-3031-3935-39361b587621,T.Brady,2.0,5.0,1.0,0.0,0.0,0.0,32013030-2d30-3031-3935-39361b587621,T.Brady,1.0,4.0,0.0,0.0,0.0,0.0,T.Brady,32013030-2d30-3031-3935-39361b587621,NaN,NaN,NaN,NaN
1,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1.0,0.0,0.0,1.0,0.0,0.0,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1.0,11.0,0.0,0.0,0.0,0.0,B.Roethlisberger,32013030-2d30-3032-3239-3234f41ff7f7,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1.0,-2.0
2,32013030-2d30-3032-3334-35395dc60da5,A.Rodgers,1.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.Rodgers,32013030-2d30-3032-3334-35395dc60da5,NaN,NaN,NaN,NaN
3,32013030-2d30-3032-3335-3030dd77cad3,F.Gore,6.0,24.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F.Gore,32013030-2d30-3032-3335-3030dd77cad3,NaN,NaN,NaN,NaN
4,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1.0,2.0,0.0,0.0,0.0,0.0,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,3.0,17.0,0.0,0.0,0.0,0.0,R.Fitzpatrick,32013030-2d30-3032-3336-3832b8755c76,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1.0,-1.0
5,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14.0,93.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.Peterson,32013030-2d30-3032-3533-3934ceb6c025,NaN,NaN,NaN,NaN
6,32013030-2d30-3032-3631-34338634c9b6,M.Ryan,1.0,-1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M.Ryan,32013030-2d30-3032-3631-34338634c9b6,NaN,NaN,NaN,NaN
7,32013030-2d30-3032-3634-39382e08c011,M.Stafford,1.0,1.0,0.0,0.0,0.0,0.0,32013030-2d30-3032-3634-39382e08c011,M.Stafford,4.0,22.0,0.0,0.0,0.0,0.0,M.Stafford,32013030-2d30-3032-3634-39382e08c011,NaN,NaN,NaN,NaN
8,32013030-2d30-3032-3731-35304d0e9eb8,J.Edelman,1.0,23.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J.Edelman,32013030-2d30-3032-3731-35304d0e9eb8,NaN,NaN,NaN,NaN
9,32013030-2d30-3032-3739-33399bbb0097,C.Newton,13.0,64.0,2.0,0.0,0.0,0.0,32013030-2d30-3032-3739-33399bbb0097,C.Newton,1.0,12.0,0.0,0.0,0.0,0.0,C.Newton,32013030-2d30-3032-3739-33399bbb0097,32013030-2d30-3032-3739-33399bbb0097,C.Newton,1.0,-1.0


## Clean Up Columns

In [190]:
df_rush_stats_all[['player_id', 'player', 'rusher', 'passer', 'rusher_kneel']]



,player_id,player,rusher,passer,rusher_kneel
0,32013030-2d30-3031-3935-39361b587621,T.Brady,T.Brady,T.Brady,NaN
1,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,B.Roethlisberger,B.Roethlisberger,B.Roethlisberger
2,32013030-2d30-3032-3334-35395dc60da5,A.Rodgers,A.Rodgers,NaN,NaN
3,32013030-2d30-3032-3335-3030dd77cad3,F.Gore,F.Gore,NaN,NaN
4,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,R.Fitzpatrick,R.Fitzpatrick,R.Fitzpatrick
5,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,A.Peterson,NaN,NaN
6,32013030-2d30-3032-3631-34338634c9b6,M.Ryan,M.Ryan,NaN,NaN
7,32013030-2d30-3032-3634-39382e08c011,M.Stafford,M.Stafford,M.Stafford,NaN
8,32013030-2d30-3032-3731-35304d0e9eb8,J.Edelman,J.Edelman,NaN,NaN
9,32013030-2d30-3032-3739-33399bbb0097,C.Newton,C.Newton,C.Newton,C.Newton


In [191]:
df_rush_stats_all.fillna(0, inplace=True)

In [192]:
def float_to_int(column):
    return np.int64(column) if column.dtype == np.float64 else column
        
df_rush_stats_all = df_rush_stats_all.apply(float_to_int)  
df_rush_stats_all

,rusher_id,rusher,rush,yards_gained_designed,rush_touchdown_designed,fumble_designed,fumble_lost_designed,fumble_out_of_bounds_designed,passer_id,passer,pass,yards_gained_scrambles,rush_touchdown_scrambles,fumble_scrambles,fumble_lost_scrambles,fumble_out_of_bounds_scrambles,player,player_id,rusher_id_kneel,rusher_kneel,qb_kneel,yards_gained
0,32013030-2d30-3031-3935-39361b587621,T.Brady,2,5,1,0,0,0,32013030-2d30-3031-3935-39361b587621,T.Brady,1,4,0,0,0,0,T.Brady,32013030-2d30-3031-3935-39361b587621,0,0,0,0
1,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1,0,0,1,0,0,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1,11,0,0,0,0,B.Roethlisberger,32013030-2d30-3032-3239-3234f41ff7f7,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1,-2
2,32013030-2d30-3032-3334-35395dc60da5,A.Rodgers,1,2,0,0,0,0,0,0,0,0,0,0,0,0,A.Rodgers,32013030-2d30-3032-3334-35395dc60da5,0,0,0,0
3,32013030-2d30-3032-3335-3030dd77cad3,F.Gore,6,24,0,0,0,0,0,0,0,0,0,0,0,0,F.Gore,32013030-2d30-3032-3335-3030dd77cad3,0,0,0,0
4,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1,2,0,0,0,0,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,3,17,0,0,0,0,R.Fitzpatrick,32013030-2d30-3032-3336-3832b8755c76,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1,-1
5,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14,93,0,0,0,0,0,0,0,0,0,0,0,0,A.Peterson,32013030-2d30-3032-3533-3934ceb6c025,0,0,0,0
6,32013030-2d30-3032-3631-34338634c9b6,M.Ryan,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,M.Ryan,32013030-2d30-3032-3631-34338634c9b6,0,0,0,0
7,32013030-2d30-3032-3634-39382e08c011,M.Stafford,1,1,0,0,0,0,32013030-2d30-3032-3634-39382e08c011,M.Stafford,4,22,0,0,0,0,M.Stafford,32013030-2d30-3032-3634-39382e08c011,0,0,0,0
8,32013030-2d30-3032-3731-35304d0e9eb8,J.Edelman,1,23,0,0,0,0,0,0,0,0,0,0,0,0,J.Edelman,32013030-2d30-3032-3731-35304d0e9eb8,0,0,0,0
9,32013030-2d30-3032-3739-33399bbb0097,C.Newton,13,64,2,0,0,0,32013030-2d30-3032-3739-33399bbb0097,C.Newton,1,12,0,0,0,0,C.Newton,32013030-2d30-3032-3739-33399bbb0097,32013030-2d30-3032-3739-33399bbb0097,C.Newton,1,-1


In [193]:
df_rush_stats_all.columns

Index(['rusher_id', 'rusher', 'rush', 'yards_gained_designed',
       'rush_touchdown_designed', 'fumble_designed', 'fumble_lost_designed',
       'fumble_out_of_bounds_designed', 'passer_id', 'passer', 'pass',
       'yards_gained_scrambles', 'rush_touchdown_scrambles',
       'fumble_scrambles', 'fumble_lost_scrambles',
       'fumble_out_of_bounds_scrambles', 'player', 'player_id',
       'rusher_id_kneel', 'rusher_kneel', 'qb_kneel', 'yards_gained'],
      dtype='object')

In [197]:
final_columns = ['player_id', 'player','rush', 'yards_gained_designed', 
       'rush_touchdown_designed', 'fumble_designed', 'fumble_lost_designed', 
       'fumble_out_of_bounds_designed', 'pass', 
       'yards_gained_scrambles', 'rush_touchdown_scrambles', 
       'fumble_scrambles', 'fumble_lost_scrambles', 
       'fumble_out_of_bounds_scrambles', 'qb_kneel', 'yards_gained']

df_rush_stats_all = df_rush_stats_all[final_columns]
df_rush_stats_all

,player_id,player,rush,yards_gained_designed,rush_touchdown_designed,fumble_designed,fumble_lost_designed,fumble_out_of_bounds_designed,pass,yards_gained_scrambles,rush_touchdown_scrambles,fumble_scrambles,fumble_lost_scrambles,fumble_out_of_bounds_scrambles,qb_kneel,yards_gained
0,32013030-2d30-3031-3935-39361b587621,T.Brady,2,5,1,0,0,0,1,4,0,0,0,0,0,0
1,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1,0,0,1,0,0,1,11,0,0,0,0,1,-2
2,32013030-2d30-3032-3334-35395dc60da5,A.Rodgers,1,2,0,0,0,0,0,0,0,0,0,0,0,0
3,32013030-2d30-3032-3335-3030dd77cad3,F.Gore,6,24,0,0,0,0,0,0,0,0,0,0,0,0
4,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1,2,0,0,0,0,3,17,0,0,0,0,1,-1
5,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14,93,0,0,0,0,0,0,0,0,0,0,0,0
6,32013030-2d30-3032-3631-34338634c9b6,M.Ryan,1,-1,0,0,0,0,0,0,0,0,0,0,0,0
7,32013030-2d30-3032-3634-39382e08c011,M.Stafford,1,1,0,0,0,0,4,22,0,0,0,0,0,0
8,32013030-2d30-3032-3731-35304d0e9eb8,J.Edelman,1,23,0,0,0,0,0,0,0,0,0,0,0,0
9,32013030-2d30-3032-3739-33399bbb0097,C.Newton,13,64,2,0,0,0,1,12,0,0,0,0,1,-1


## Load in the roster data

In [198]:
ROSTER_PATH = os.path.join(DATA_DIRECTORY, "roster", 'roster.csv')

df_roster = pd.read_csv(ROSTER_PATH)
df_roster.columns = [col.replace(".", '_') if '.' in col else col for col in df_roster.columns]
df_roster.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,team_season,team_abbr,teamPlayers_jerseyNumber,teamPlayers_displayName,teamPlayers_firstName,teamPlayers_middleName,teamPlayers_lastName,teamPlayers_suffix,teamPlayers_status,teamPlayers_positionGroup,teamPlayers_position,teamPlayers_nflId,teamPlayers_esbId,teamPlayers_gsisId,teamPlayers_birthDate,teamPlayers_homeTown,teamPlayers_collegeId,teamPlayers_collegeName,teamPlayers_height,teamPlayers_weight,team_teamId,team_cityState,team_fullName,team_nick,team_conferenceAbbr,team_divisionAbbr,teamPlayers_headshot_url,teamPlayers_profile_url,scrape_dt,join,pbp_id,pbp_name
0,1999,ARI,24.0,Mario Bates,Mario,Doniel,Bates,NaN,NaN,RB,RB,2499570.0,BAT111777,00-0000871,1973-01-16,"Tucson, AZ",4007.0,Arizona State,6-2,237.0,3800.0,Arizona,Arizona Cardinals,Cardinals,NFC,NCE,http://static.nfl.com/static/content/public/st...,http://www.nfl.com/player/mariobates/2499570/p...,NaN,Bates,NaN,NaN
1,1999,ARI,28.0,Tommy Bennett,Tommy,NaN,Bennett,NaN,NaN,DB,SS,2499626.0,BEN447808,00-0001114,1973-02-19,"Las Vegas, NV",4837.0,UCLA,6-2,212.0,3800.0,Arizona,Arizona Cardinals,Cardinals,NFC,NCE,http://static.nfl.com/static/content/public/st...,http://www.nfl.com/player/tommybennett/2499626...,NaN,Bennett,NaN,NaN
2,1999,ARI,85.0,Rob Moore,Rob,Sean,Moore,NaN,NaN,WR,WR,2502190.0,MOO663262,00-0011601,1968-09-27,"New York, NY",2823.0,Syracuse,6-3,202.0,3800.0,Arizona,Arizona Cardinals,Cardinals,NFC,NCE,http://static.nfl.com/static/content/public/st...,http://www.nfl.com/player/robmoore/2502190/pro...,NaN,Moore,NaN,NaN
3,1999,ARI,74.0,Mike Moten,Mike,Edward,Moten,NaN,NaN,DL,DT,2502224.0,MOT318011,00-0011764,1974-03-12,"Daytona Beach, FL",5812.0,Florida,6-5,266.0,3800.0,Arizona,Arizona Cardinals,Cardinals,NFC,NCE,http://static.nfl.com/static/content/public/st...,http://www.nfl.com/player/mikemoten/2502224/pr...,NaN,Moten,NaN,NaN
4,1999,ARI,29.0,Adrian Murrell,Adrian,Bryan,Murrell,NaN,NaN,RB,RB,2502245.0,MUR719159,00-0011869,1970-10-16,"Fayetteville, NC",5904.0,West Virginia,5-11,211.0,3800.0,Arizona,Arizona Cardinals,Cardinals,NFC,NCE,http://static.nfl.com/static/content/public/st...,http://www.nfl.com/player/adrianmurrell/250224...,NaN,Murrell,NaN,NaN


In [111]:
df_roster['team_season'].value_counts()

2018    3144
2019    3112
2011    3098
2017    3089
2016    3059
2015    3054
2014    3029
2013    3024
2012    2997
2006    2947
2007    2931
2020    2804
2008    2762
2010    2758
2009    2712
2005    2150
2004    2150
2003    2105
2002    2104
2001    1988
2000    1988
1999    1980
Name: team_season, dtype: int64

In [200]:
df_roster_2020 = df_roster.loc[df_roster['team_season'] == 2020]
df_roster_2020

,team_season,team_abbr,teamPlayers_jerseyNumber,teamPlayers_displayName,teamPlayers_firstName,teamPlayers_middleName,teamPlayers_lastName,teamPlayers_suffix,teamPlayers_status,teamPlayers_positionGroup,teamPlayers_position,teamPlayers_nflId,teamPlayers_esbId,teamPlayers_gsisId,teamPlayers_birthDate,teamPlayers_homeTown,teamPlayers_collegeId,teamPlayers_collegeName,teamPlayers_height,teamPlayers_weight,team_teamId,team_cityState,team_fullName,team_nick,team_conferenceAbbr,team_divisionAbbr,teamPlayers_headshot_url,teamPlayers_profile_url,scrape_dt,join,pbp_id,pbp_name
56181,2020,ARI,22.0,Bejour Wilson,Bejour,NaN,Wilson,NaN,ACT,NaN,CB,NaN,NaN,NaN,NaN,NaN,NaN,Liberty,5-10,170.0,NaN,NaN,Arizona Cardinals,NaN,NaN,NaN,NaN,NaN,2020-09-10T06:01:05Z,Wilson,NaN,NaN
56182,2020,ARI,87.0,Maxx Williams,Maxx,NaN,Williams,NaN,ACT,NaN,TE,NaN,NaN,NaN,1994-04-12,NaN,NaN,Minnesota,6-4,252.0,NaN,NaN,Arizona Cardinals,NaN,NaN,NaN,NaN,NaN,2020-09-10T06:01:05Z,Williams,32013030-2d30-3033-3135-35387ca5d707,M.Williams
56183,2020,ARI,39.0,Jace Whittaker,Jace,NaN,Whittaker,NaN,DEV,NaN,CB,NaN,NaN,NaN,1995-07-16,NaN,NaN,Arizona,5-11,185.0,NaN,NaN,Arizona Cardinals,NaN,NaN,NaN,NaN,NaN,2020-09-10T06:01:05Z,Whittaker,NaN,NaN
56184,2020,ARI,50.0,Evan Weaver,Evan,NaN,Weaver,NaN,DEV,NaN,LB,NaN,NaN,NaN,1998-08-11,NaN,NaN,California,6-3,235.0,NaN,NaN,Arizona Cardinals,NaN,NaN,NaN,NaN,NaN,2020-09-10T06:01:05Z,Weaver,NaN,NaN
56185,2020,ARI,28.0,Charles Washington,Charles,NaN,Washington,NaN,ACT,NaN,SAF,NaN,NaN,NaN,1993-03-10,NaN,NaN,Fresno State,5-10,192.0,NaN,NaN,Arizona Cardinals,NaN,NaN,NaN,NaN,NaN,2020-09-10T06:01:05Z,Washington,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58980,2020,WAS,80.0,Dontrelle Inman,Dontrelle,NaN,Inman,NaN,ACT,NaN,WR,NaN,NaN,NaN,1989-01-31,NaN,NaN,Virginia,6-3,205.0,NaN,NaN,Washington Football Team,NaN,NaN,NaN,NaN,NaN,2020-09-17T06:00:18Z,Inman,32013030-2d30-3032-3834-3131ecbba609,D.Inman
58981,2020,WAS,29.0,Kendall Fuller,Kendall,NaN,Fuller,NaN,ACT,NaN,CB,NaN,NaN,NaN,1995-02-13,NaN,NaN,Virginia Tech,5-11,198.0,NaN,NaN,Washington Football Team,NaN,NaN,NaN,NaN,NaN,2020-09-17T06:00:18Z,Fuller,NaN,NaN
58982,2020,WAS,23.0,Ronald Darby,Ronald,NaN,Darby,NaN,ACT,NaN,CB,NaN,NaN,NaN,1994-01-02,NaN,NaN,Florida State,5-11,193.0,NaN,NaN,Washington Football Team,NaN,NaN,NaN,NaN,NaN,2020-09-17T06:00:18Z,Darby,NaN,NaN
58983,2020,WAS,26.0,Landon Collins,Landon,NaN,Collins,NaN,ACT,NaN,SS,NaN,NaN,NaN,1994-01-10,NaN,NaN,Alabama,6-0,218.0,NaN,NaN,Washington Football Team,NaN,NaN,NaN,NaN,NaN,2020-09-17T06:00:18Z,Collins,NaN,NaN


In [201]:
df_roster_2020['pbp_name'].value_counts()

D.Johnson       4
M.Thomas        3
T.Taylor        3
D.Harris        3
D.Washington    2
               ..
M.Evans         1
J.Burrow        1
J.Wilson        1
G.Bradbury      1
J.Hollister     1
Name: pbp_name, Length: 647, dtype: int64

## Join the Roster data

In [258]:
df_roster_2020.rename(columns={
    'pbp_name': 'rusher',
    'teamPlayers_position': 'pos',
    'team_abbr' : 'team'
}, inplace=True)
cols = ['team', 'pos', 'pbp_id']
df_rush_stats_all_position = df_rush_stats_all.merge(df_roster_2020[cols], how='left', left_on='player_id', right_on='pbp_id')

assert(sum(df_rush_stats_all_position['pos'].isna()) == 0)
df_rush_stats_all_position

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,player_id,player,rush,yards_gained_designed,rush_touchdown_designed,fumble_designed,fumble_lost_designed,fumble_out_of_bounds_designed,pass,yards_gained_scrambles,rush_touchdown_scrambles,fumble_scrambles,fumble_lost_scrambles,fumble_out_of_bounds_scrambles,qb_kneel,yards_gained,team,pos,pbp_id
0,32013030-2d30-3031-3935-39361b587621,T.Brady,2,5,1,0,0,0,1,4,0,0,0,0,0,0,TB,QB,32013030-2d30-3031-3935-39361b587621
1,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1,0,0,1,0,0,1,11,0,0,0,0,1,-2,PIT,QB,32013030-2d30-3032-3239-3234f41ff7f7
2,32013030-2d30-3032-3334-35395dc60da5,A.Rodgers,1,2,0,0,0,0,0,0,0,0,0,0,0,0,GB,QB,32013030-2d30-3032-3334-35395dc60da5
3,32013030-2d30-3032-3335-3030dd77cad3,F.Gore,6,24,0,0,0,0,0,0,0,0,0,0,0,0,NYJ,RB,32013030-2d30-3032-3335-3030dd77cad3
4,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1,2,0,0,0,0,3,17,0,0,0,0,1,-1,MIA,QB,32013030-2d30-3032-3336-3832b8755c76
5,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14,93,0,0,0,0,0,0,0,0,0,0,0,0,DET,RB,32013030-2d30-3032-3533-3934ceb6c025
6,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14,93,0,0,0,0,0,0,0,0,0,0,0,0,DET,RB,32013030-2d30-3032-3533-3934ceb6c025
7,32013030-2d30-3032-3631-34338634c9b6,M.Ryan,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,ATL,QB,32013030-2d30-3032-3631-34338634c9b6
8,32013030-2d30-3032-3634-39382e08c011,M.Stafford,1,1,0,0,0,0,4,22,0,0,0,0,0,0,DET,QB,32013030-2d30-3032-3634-39382e08c011
9,32013030-2d30-3032-3731-35304d0e9eb8,J.Edelman,1,23,0,0,0,0,0,0,0,0,0,0,0,0,NE,WR,32013030-2d30-3032-3731-35304d0e9eb8


## Add Columns

In [259]:
df_rush_stats_all_position.rename(columns={
    'qb_kneel': 'qb_kneels',
    'yards_gained': 'qb_kneel_yards',
    'pass': 'scrambles',
    'rush_touchdown_scrambles': 'scramble_TDs',
    'yards_gained_scrambles': 'scramble_yards'
}, inplace=True)

In [265]:
df_rush_stats_all_position['total_yards'] = df_rush_stats_all_position['yards_gained_designed'] + \
    df_rush_stats_all_position['scramble_yards'] + df_rush_stats_all_position['qb_kneel_yards'] 

df_rush_stats_all_position['total_attempts'] = df_rush_stats_all_position['rush'] + \
    df_rush_stats_all_position['scrambles'] + df_rush_stats_all_position['qb_kneels'] 

df_rush_stats_all_position['TDs'] = df_rush_stats_all_position['rush_touchdown_designed'] + \
    df_rush_stats_all_position['scramble_TDs']
    
df_rush_stats_all_position['fumbles'] = df_rush_stats_all_position['fumble_designed'] + \
    df_rush_stats_all_position['fumble_scrambles']
    
df_rush_stats_all_position['fumbles_lost'] = df_rush_stats_all_position['fumble_lost_designed'] + \
    df_rush_stats_all_position['fumble_lost_scrambles']
    
df_rush_stats_all_position['fumbles_out_of_bounds'] = df_rush_stats_all_position['fumble_out_of_bounds_designed'] + \
    df_rush_stats_all_position['fumble_out_of_bounds_scrambles']

In [266]:
final_rb_columns = ['year', 'week', 'player_id', 'team', 'pos', 'player',
    'total_attempts', 'total_yards', 'TDs', 'fumbles', 'fumbles_lost', 'fumbles_out_of_bounds'
]

mask = df_rush_stats_all_position['pos'] == 'RB'
df_rush_stats_all_position[mask][final_rb_columns].sort_values('total_yards', ascending=False)

,player_id,team,pos,player,total_attempts,total_yards,TDs,fumbles,fumbles_lost,fumbles_out_of_bounds
127,32013030-2d30-3033-3633-3630b28b4868,KC,RB,C.Edwards-Helaire,25,138,1,0,0,0
43,32013030-2d30-3033-3237-3634214e1686,TEN,RB,D.Henry,31,116,0,0,0,0
99,32013030-2d30-3033-3532-3137ad078b2e,PIT,RB,B.Snell,19,113,0,1,0,0
50,32013030-2d30-3033-3332-3830ec2c3be7,CAR,RB,C.McCaffrey,23,96,2,0,0,0
47,32013030-2d30-3033-3330-343523cdc4a0,DAL,RB,E.Elliott,22,96,1,0,0,0
5,32013030-2d30-3032-3533-3934ceb6c025,DET,RB,A.Peterson,14,93,0,0,0,0
6,32013030-2d30-3032-3533-3934ceb6c025,DET,RB,A.Peterson,14,93,0,0,0,0
111,32013030-2d30-3033-3537-3030adfaf832,LV,RB,J.Jacobs,25,93,3,0,0,0
64,32013030-2d30-3033-3336-39396b8ab7d5,LAC,RB,A.Ekeler,19,84,0,0,0,0
31,32013030-2d30-3033-3138-30361fd87e06,LA,RB,M.Brown,18,79,2,0,0,0


In [267]:
final_qb_columns = ['player_id', 'team', 'pos', 'player',
    'total_attempts', 'total_yards', 'TDs', 'scrambles', 'scramble_yards', 'scramble_TDs', 'qb_kneels', 'qb_kneel_yards',
    'fumbles', 'fumbles_lost', 'fumbles_out_of_bounds',
                    
]

mask = df_rush_stats_all_position['pos'] == 'QB'
df_rush_stats_all_position[mask][final_qb_columns].sort_values('total_yards', ascending=False)

,player_id,team,pos,player,total_attempts,total_yards,TDs,scrambles,scramble_yards,scramble_TDs,qb_kneels,qb_kneel_yards,fumbles,fumbles_lost,fumbles_out_of_bounds
100,32013030-2d30-3033-3532-323871689b29,ARI,QB,K.Murray,13,91,1,7,90,1,2,-9,0,0,0
10,32013030-2d30-3032-3739-33399bbb0097,NE,QB,C.Newton,15,75,2,1,12,0,1,-1,0,0,0
94,32013030-2d30-3033-3438-3537f3a13a66,BUF,QB,J.Allen,14,57,1,3,28,0,2,-2,2,2,0
130,32013030-2d30-3033-3634-3432d92fc82e,CIN,QB,J.Burrow,8,46,1,4,15,0,0,0,1,0,1
88,32013030-2d30-3033-3437-39369c80073a,BAL,QB,L.Jackson,7,45,0,2,22,0,0,0,1,0,0
133,32013030-2d30-3032-3936-303492e9d55e,MIN,QB,K.Cousins,4,34,0,4,34,0,0,0,0,0,0
136,32013030-2d30-3033-3330-37372d13d088,DAL,QB,D.Prescott,3,30,0,3,30,0,0,0,0,0,0
15,32013030-2d30-3032-3932-363323eefb5c,SEA,QB,R.Wilson,3,29,0,1,2,0,1,-1,0,0,0
58,32013030-2d30-3033-3335-33372bbab8b8,HOU,QB,D.Watson,6,27,1,4,27,0,1,-1,0,0,0
134,32013030-2d30-3033-3132-3337a759b614,CAR,QB,T.Bridgewater,4,26,0,4,26,0,0,0,0,0,0


## Get Team Breakdown

In [272]:
df_team_rushing_totals = df_rush_stats_all_position.groupby(['team'], as_index=False).sum()
df_team_rushing_totals.rename(columns={
    col: 'team_' + col for col in df_team_rushing_totals.columns if col != 'team'
}, inplace=True)
df_team_rushing_totals

,team,team_rush,team_yards_gained_designed,team_rush_touchdown_designed,team_fumble_designed,team_fumble_lost_designed,team_fumble_out_of_bounds_designed,team_scrambles,team_scramble_yards,team_scramble_TDs,team_fumble_scrambles,team_fumble_lost_scrambles,team_fumble_out_of_bounds_scrambles,team_qb_kneels,team_qb_kneel_yards,team_total_yards,team_TDs,team_fumbles,team_fumbles_lost,team_fumbles_out_of_bounds,team_total__attempts,team_total_attempts
0,ARI,27,99,1,1,0,0,7,90,1,0,0,0,2,-9,180,2,1,0,0,36,36
1,ATL,21,72,1,1,1,0,0,0,0,0,0,0,0,0,72,1,1,1,0,21,21
2,BAL,27,86,2,2,1,0,2,22,0,0,0,0,1,-1,107,2,2,1,0,30,30
3,BUF,27,72,1,2,2,0,3,28,0,0,0,0,2,-2,98,1,2,2,0,32,32
4,CAR,26,102,2,0,0,0,4,26,0,0,0,0,0,0,128,2,0,0,0,30,30
5,CHI,26,123,0,0,0,0,2,26,0,0,0,0,0,0,149,0,0,0,0,28,28
6,CIN,24,107,1,2,1,1,4,15,0,0,0,0,0,0,122,1,2,1,1,28,28
7,CLE,25,135,0,3,2,0,1,4,0,0,0,0,1,-1,138,0,3,2,0,27,27
8,DAL,24,106,1,0,0,0,3,30,0,0,0,0,0,0,136,1,0,0,0,27,27
9,DEN,25,104,1,2,0,0,1,3,0,0,0,0,0,0,107,1,2,0,0,26,26


In [274]:
join_cols = ['team', 'team_rush', 'team_yards_gained_designed']
df_rush_stats_all_position = df_rush_stats_all_position.merge(df_team_rushing_totals[join_cols], how='left', on='team')
df_rush_stats_all_position


,player_id,player,rush,yards_gained_designed,rush_touchdown_designed,fumble_designed,fumble_lost_designed,fumble_out_of_bounds_designed,scrambles,scramble_yards,scramble_TDs,fumble_scrambles,fumble_lost_scrambles,fumble_out_of_bounds_scrambles,qb_kneels,qb_kneel_yards,team,pos,pbp_id,total_yards,TDs,fumbles,fumbles_lost,fumbles_out_of_bounds,total__attempts,total_attempts,team_rush,team_yards_gained_designed
0,32013030-2d30-3031-3935-39361b587621,T.Brady,2,5,1,0,0,0,1,4,0,0,0,0,0,0,TB,QB,32013030-2d30-3031-3935-39361b587621,9,1,0,0,0,3,3,30,87
1,32013030-2d30-3032-3239-3234f41ff7f7,B.Roethlisberger,1,0,0,1,0,0,1,11,0,0,0,0,1,-2,PIT,QB,32013030-2d30-3032-3239-3234f41ff7f7,9,0,1,0,0,3,3,28,132
2,32013030-2d30-3032-3334-35395dc60da5,A.Rodgers,1,2,0,0,0,0,0,0,0,0,0,0,0,0,GB,QB,32013030-2d30-3032-3334-35395dc60da5,2,0,0,0,0,1,1,30,160
3,32013030-2d30-3032-3335-3030dd77cad3,F.Gore,6,24,0,0,0,0,0,0,0,0,0,0,0,0,NYJ,RB,32013030-2d30-3032-3335-3030dd77cad3,24,0,0,0,0,6,6,15,52
4,32013030-2d30-3032-3336-3832b8755c76,R.Fitzpatrick,1,2,0,0,0,0,3,17,0,0,0,0,1,-1,MIA,QB,32013030-2d30-3032-3336-3832b8755c76,18,0,0,0,0,5,5,23,71
5,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14,93,0,0,0,0,0,0,0,0,0,0,0,0,DET,RB,32013030-2d30-3032-3533-3934ceb6c025,93,0,0,0,0,14,14,42,217
6,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14,93,0,0,0,0,0,0,0,0,0,0,0,0,DET,RB,32013030-2d30-3032-3533-3934ceb6c025,93,0,0,0,0,14,14,42,217
7,32013030-2d30-3032-3631-34338634c9b6,M.Ryan,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,ATL,QB,32013030-2d30-3032-3631-34338634c9b6,-1,0,0,0,0,1,1,21,72
8,32013030-2d30-3032-3634-39382e08c011,M.Stafford,1,1,0,0,0,0,4,22,0,0,0,0,0,0,DET,QB,32013030-2d30-3032-3634-39382e08c011,23,0,0,0,0,5,5,42,217
9,32013030-2d30-3032-3731-35304d0e9eb8,J.Edelman,1,23,0,0,0,0,0,0,0,0,0,0,0,0,NE,WR,32013030-2d30-3032-3731-35304d0e9eb8,23,0,0,0,0,1,1,40,206


In [277]:
df_rush_stats_all_position['%_team_carries'] = df_rush_stats_all_position['rush'] / \
    df_rush_stats_all_position['team_rush']
df_rush_stats_all_position['%_team_yards'] = df_rush_stats_all_position['yards_gained_designed'] / \
    df_rush_stats_all_position['team_yards_gained_designed']

df_rush_stats_all_position.sort_values('yards_gained_designed', ascending=False)


,player_id,player,rush,yards_gained_designed,rush_touchdown_designed,fumble_designed,fumble_lost_designed,fumble_out_of_bounds_designed,scrambles,scramble_yards,scramble_TDs,fumble_scrambles,fumble_lost_scrambles,fumble_out_of_bounds_scrambles,qb_kneels,qb_kneel_yards,team,pos,pbp_id,total_yards,TDs,fumbles,fumbles_lost,fumbles_out_of_bounds,total__attempts,total_attempts,team_rush,team_yards_gained_designed,%_team_carries,%_team_yards
127,32013030-2d30-3033-3633-3630b28b4868,C.Edwards-Helaire,25,138,1,0,0,0,0,0,0,0,0,0,0,0,KC,RB,32013030-2d30-3033-3633-3630b28b4868,138,1,0,0,0,25,25,34,166,0.735294,0.831325
43,32013030-2d30-3033-3237-3634214e1686,D.Henry,31,116,0,0,0,0,0,0,0,0,0,0,0,0,TEN,RB,32013030-2d30-3033-3237-3634214e1686,116,0,0,0,0,31,31,33,119,0.939394,0.974790
99,32013030-2d30-3033-3532-3137ad078b2e,B.Snell,19,113,0,1,0,0,0,0,0,0,0,0,0,0,PIT,RB,32013030-2d30-3033-3532-3137ad078b2e,113,0,1,0,0,19,19,28,132,0.678571,0.856061
47,32013030-2d30-3033-3330-343523cdc4a0,E.Elliott,22,96,1,0,0,0,0,0,0,0,0,0,0,0,DAL,RB,32013030-2d30-3033-3330-343523cdc4a0,96,1,0,0,0,22,22,24,106,0.916667,0.905660
50,32013030-2d30-3033-3332-3830ec2c3be7,C.McCaffrey,23,96,2,0,0,0,0,0,0,0,0,0,0,0,CAR,RB,32013030-2d30-3033-3332-3830ec2c3be7,96,2,0,0,0,23,23,26,102,0.884615,0.941176
111,32013030-2d30-3033-3537-3030adfaf832,J.Jacobs,25,93,3,0,0,0,0,0,0,0,0,0,0,0,LV,RB,32013030-2d30-3033-3537-3030adfaf832,93,3,0,0,0,25,25,31,133,0.806452,0.699248
5,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14,93,0,0,0,0,0,0,0,0,0,0,0,0,DET,RB,32013030-2d30-3032-3533-3934ceb6c025,93,0,0,0,0,14,14,42,217,0.333333,0.428571
6,32013030-2d30-3032-3533-3934ceb6c025,A.Peterson,14,93,0,0,0,0,0,0,0,0,0,0,0,0,DET,RB,32013030-2d30-3032-3533-3934ceb6c025,93,0,0,0,0,14,14,42,217,0.333333,0.428571
64,32013030-2d30-3033-3336-39396b8ab7d5,A.Ekeler,19,84,0,0,0,0,0,0,0,0,0,0,0,0,LAC,RB,32013030-2d30-3033-3336-39396b8ab7d5,84,0,0,0,0,19,19,36,149,0.527778,0.563758
31,32013030-2d30-3033-3138-30361fd87e06,M.Brown,18,79,2,0,0,0,0,0,0,0,0,0,0,0,LA,RB,32013030-2d30-3033-3138-30361fd87e06,79,2,0,0,0,18,18,37,146,0.486486,0.541096
